In [32]:
import pandas as pd
import re

In [34]:
df = pd.read_csv("ikea_reviews_multi.csv")

In [36]:
# Split the itemKey dictionary string into three separate columns using regex
item_pattern = r"\{'itemNo': '([^']+)', 'itemType': '([^']+)', 'sourceItemNo': '([^']+)'\}"
df[["itemNo", "itemType", "sourceItemNo"]] = df["itemKey"].str.extract(item_pattern)

df = df.drop(columns=["itemKey"])

In [38]:
# Extract the numeric ratingValue from the primaryRating text using regex
df["primaryRatingValue"] = df["primaryRating"].str.extract(
    r"'ratingValue':\s*([0-9]+)"
).astype(float)

df = df.drop(columns=["primaryRating"])

In [40]:
# Extract the ratingValue for a given secondary rating id using regex
def get_secondary_rating(s, sec_id):
    if pd.isna(s):
        return None
    
    # Match 'ratingValue': X ... 'id': 'sec_id'
    # BUT do not cross into another block that has a new 'ratingValue'
    pattern = rf"'ratingValue':\s*([0-9]+)(?:(?!'ratingValue').)*'id':\s*'{sec_id}'"
    m = re.search(pattern, s, flags=re.S)
    return float(m.group(1)) if m else None


sec_ids = ["appearance", "product_quality", "value_for_money", "works_as_expected"]

for sec_id in sec_ids:
    col_name = "sec_" + sec_id
    df[col_name] = df["secondaryRatings"].apply(lambda s: get_secondary_rating(s, sec_id))

df = df.drop(columns=["secondaryRatings"])

In [42]:
# Extract the reviewer's displayName from the reviewer text using regex
df["reviewerName"] = df["reviewer"].str.extract(
    r"'displayName': '([^']*)'"
)

df = df.drop(columns=["reviewer"])

In [44]:
df["submissionOn"] = pd.to_datetime(df["submissionOn"])

In [46]:
df["updatedOn"] = pd.to_datetime(df["updatedOn"])

In [48]:
df = df.drop(columns=["id", "hasMedia", "medias", "response", "itemNo", "sourceItemNo"])

In [50]:
df.iloc[85]

productId                                                          70511903
type                                                                    web
sourceCountryCode                                                        us
sourceLangCode                                                           en
title                               Ikea refuses to follow their own policy
text                      Bought mattress and then it came on sale a wee...
isRecommended                                                          True
verifiedPurchase                                                      False
verifiedReviewer                                                       True
anonymousReviewer                                                     False
positiveFeedbacksCount                                                   43
negativeFeedbacksCount                                                    0
submissionOn                                      2025-02-12 03:45:37+00:00
updatedOn   

In [52]:
primary = 'primaryRatingValue'
secondary_cols = [
    'sec_appearance',
    'sec_product_quality',
    'sec_value_for_money',
    'sec_works_as_expected'
]

In [54]:
# 1. average secondary rating per review (ignore NaNs)
df['sec_mean'] = df[secondary_cols].mean(axis=1, skipna=True)

# 2. difference between secondary mean and primary rating
df['sec_primary_diff'] = df['sec_mean'] - df['primaryRatingValue']
df['sec_primary_abs_diff'] = df['sec_primary_diff'].abs()

# 3. choose a threshold for difference >= 2 points
threshold = 2.0

# ignore rows with no secondary rating at all
mismatched = df[df['sec_mean'].notna() & (df['sec_primary_abs_diff'] >= threshold)]

print(f"Number of reviews with |secondary - primary| >= {threshold}: {len(mismatched)}")

mismatched_display = mismatched[
    [primary, 'sec_mean', 'sec_primary_diff', 'sec_primary_abs_diff'] + secondary_cols
].head(20)

mismatched_display

Number of reviews with |secondary - primary| >= 2.0: 12


,primaryRatingValue,sec_mean,sec_primary_diff,sec_primary_abs_diff,sec_appearance,sec_product_quality,sec_value_for_money,sec_works_as_expected
85,1.0,3.00,2.00,2.00,5.0,3.0,1.0,3.0
136,3.0,5.00,2.00,2.00,5.0,5.0,5.0,5.0
300,1.0,3.25,2.25,2.25,5.0,5.0,2.0,1.0
328,2.0,4.00,2.00,2.00,5.0,4.0,5.0,2.0
538,1.0,3.50,2.50,2.50,5.0,4.0,4.0,1.0
575,5.0,3.00,-2.00,2.00,4.0,4.0,2.0,2.0
604,1.0,4.00,3.00,3.00,5.0,5.0,5.0,1.0
673,1.0,3.00,2.00,2.00,NaN,3.0,3.0,NaN
814,2.0,4.00,2.00,2.00,4.0,4.0,4.0,4.0
821,5.0,1.00,-4.00,4.00,NaN,NaN,1.0,NaN


In [56]:
df.to_csv("ikea_cleaned.csv")

In [58]:
len(df)

1156